<a href="https://colab.research.google.com/github/Nqam/Moon_report/blob/main/report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Импортируем библиотеки для работы с таблицами

In [1]:
import pandas as pd
import numpy as np
import re

In [ ]:
#Указываем путь на отчеты для каждой переменной - розничный и фабричный
#(можно загрузить файлы во временную библиотеку блокнота и провести все операции с ними)
rozn = '/content/drive/MyDrive/Розничный отчет Май 1-31.xlsx'
fabr = '/content/drive/MyDrive/Май 1-31 фабричный.xlsx'

In [73]:
#Указываем путь на отчеты для каждой переменной - розничный и фабричный
#(можно загрузить файлы во временную библиотеку блокнота и провести все операции с ними)
rozn = '/content/Июль_1_31_23г_Розничный_отчет_не_вычищенный.xls'
fabr = '/content/Июль_1_31_23г_Фабричный_отчет_не_вычищенный.xlsx'

Загружаем отчеты в память программы

In [74]:
%%time
rozn_t = pd.read_excel(rozn, skiprows=2) #это розничный отчет, пропускаются две строки, там шапка с объединенными ячейками
fabr_t = pd.read_excel(fabr) #это фабричный отчет

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


CPU times: user 12 s, sys: 29.7 ms, total: 12 s
Wall time: 12.2 s


Переименовываем все столбцы к общему виду

In [75]:
rozn_t = rozn_t.rename(columns={'Модель': 'Дизайн',
                        'Цвет':'Цвет ткани',
                        'Название акции':'Акции',
                        'Структура на момент продажи':'Коллекция'})

fabr_t = fabr_t.rename(columns={'№':'Договор',
                        'от даты':'Дата',
                        'Основа':'Ткань',
                        'Оформил':'Продавец',
                        'Канал сбыта':'Канал продаж',
                        'Цвет основы':'Цвет ткани',
                        'Модель':'Дизайн',
                        'Название акции':'Акции',
                        'Заказчик':'Розничная точка',
                        '№ комп':'Номер комплекта'
                        })

#Добавляем недостающие для слияния столбцы
rozn_t['Источник_строк'] = 'Розничный отчет'
rozn_t['Канал продаж'] = ''
rozn_t['Скид. пост.'] = np.NaN
rozn_t['Скид. фабр.'] = np.NaN
rozn_t['Кат.'] = ''
rozn_t['Номер комплекта'] = ''

fabr_t['Источник_строк'] = 'Фабричный отчет'
fabr_t['Заказчик'] = ''
fabr_t['Механизм трансформации'] = ''
fabr_t['Банковские продукты'] = ''
fabr_t['Название салона'] = ''
fabr_t['Юр. лицо'] = ''

In [76]:
rozn_t = rozn_t[[
  'Источник_строк','Акции','Банковские продукты','Бренд','Дата','Дизайн','Договор','Заказчик',
  'Канал продаж','Кат.','Категория заказчика','Комплектация','Номер комплекта','Коллекция','Механизм трансформации',
  'Название салона','Особые отметки','Пл.отгруз','Примечание','Продавец','Регион','Розничная точка',
  'Руководитель группы','Скид. пост.','Скид. прод.','Скид. фабр.','Сумма','Ткань','Цвет ткани',
  'Цена','Шт.','Юр. лицо']]

In [77]:
fabr_t = fabr_t[[
  'Источник_строк','Акции','Банковские продукты','Бренд','Дата','Дизайн','Договор',
  'Заказчик','Канал продаж','Кат.','Категория заказчика','Комплектация','Номер комплекта','Коллекция',
  'Механизм трансформации','Название салона','Особые отметки','Пл.отгруз','Примечание',
  'Продавец','Регион','Розничная точка','Руководитель группы','Скид. пост.','Скид. прод.',
  'Скид. фабр.','Сумма','Ткань','Цвет ткани','Цена','Шт.','Юр. лицо']]

In [78]:
def rozn(x):
  if x['Розничная точка'] == '' or pd.isna(x['Розничная точка']) == True:
    return ''
  elif 'оптовый' in x['Категория заказчика'].lower():
    return 'Оптовый'
  elif x['Категория заказчика'].lower() == 'франчайзи м':
    return 'Франчайзи М'
  elif x['Категория заказчика'].lower() == 'франчайзи мт':
    return 'Франчайзи MF'
  elif 'дисконт moon-trade' in x['Розничная точка'].lower():
    return 'Выставочный зал фабрики MF'
  elif 'кроват' in x['Розничная точка'].lower():
    return 'КиМ'
  elif 'интернет' not in x['Розничная точка'].lower() and 'выставочный зал ногинск' in x['Розничная точка'].lower():
    return 'Выставочный зал фабрики MOON'
  elif 'михайлова' in x['Руководитель группы'].lower() and 'дисконт moon-trade' not in x['Розничная точка'].lower():
    return 'Розница MF'
  elif 'носкова' in x['Руководитель группы'].lower() and 'дисконт moon-trade' not in x['Розничная точка'].lower():
    return 'Розница M'
  else:
    return x['Розничная точка']

rozn_t['Канал продаж'] = rozn_t.apply(rozn, axis = 1)

In [79]:
rozn_t['Канал продаж'].unique()

array(['Франчайзи М', 'Выставочный зал фабрики MF', 'Розница MF',
       'Выставочный зал фабрики MOON', 'Розница M', 'КиМ', 'Оптовый',
       'Франчайзи MF', 'Выставочный зал Ногинск(интернет)', ''],
      dtype=object)

In [80]:
def internet(x):
  if x['Розничная точка'] == '' or pd.isna(x['Розничная точка']) == True:
    return x['Канал продаж']
  elif 'живые диваны' in x['Розничная точка'].lower() and '(интернет)' in x['Розничная точка'].lower():
    return 'ИМ Фабрики (Москва)'
  elif '(интернет)' in x['Розничная точка'].lower() and 'г.' in x['Розничная точка'].lower():
    return 'ИМ Регионы'
  elif 'живые диваны' and ('ИП' in x['Розничная точка'] or x['Розничная точка'].lower().count('ооо') > 1):
    return 'ИМ Партнеры'
  else:
    return 'DROP'


fabr_t['Канал продаж'] = fabr_t.apply(internet, axis = 1)
fabr_t = fabr_t[(fabr_t['Канал продаж'] != 'DROP')&(fabr_t['Канал продаж'].isna() == False)]

In [81]:
fabr_t['Канал продаж'].unique()

array(['ИМ Партнеры', 'ИМ Регионы', 'ИМ Фабрики (Москва)'], dtype=object)

In [82]:
df = pd.concat([rozn_t,fabr_t], ignore_index=True)
df = df[df['Дата'].isna() == False]
df['Шт. для MOON'] = ''
df['Диз+мех'] = ''
df['Спальное место'] = ''
df['Тип дивана'] = ''
df['Настил'] = ''
df['Категория заказчика'] = df['Категория заказчика'].astype(str)
df[['Комплектация', 'Механизм трансформации','Розничная точка']] = df[['Комплектация', 'Механизм трансформации','Розничная точка']].fillna('')
df['Скид. прод.'] = df['Скид. прод.'].str.replace(',', '.').astype('float64')
df['Сумма'] = df['Сумма'].astype(float)
df['Шт.'] = df['Шт.'].astype(int)

In [83]:
df['Скид. прод.'].unique()

array([ 0.  , 56.41, 81.  , ..., 31.74, 66.92, 49.16])

Удаляем модель 500 и точки содержащие "склад" и "интернет"

In [84]:
def int_skl(x):
  if x == '':
    return x
  elif 'склад' in x.lower():
    return 'DROP_ROW'
  elif 'интернет' in x.lower() and 'ногинск' not in x.lower():
    return 'DROP_ROW'
  else:
    return x

df['Розничная точка_code'] = df['Розничная точка'].apply(int_skl)
df = df[(df['Розничная точка'] != 'DROP_ROW')&(df['Дизайн'] != '500')]
df = df.drop('Розничная точка_code', axis=1)

Меняем поле "дата" на нормальный тип

In [85]:
#from pandas.io.sql import date
def str_to_date(x):
  x = '.'.join([x[0:2],x[3:5],x[6:10]])
  return x

df['Дата'] = pd.to_datetime(df['Дата'].apply(str_to_date), dayfirst=True)

Убираем лишнее в поле "механизм трансформации" (МТ, кавычки)

In [86]:
def mechanism(x):
  if x == '':
    return x
  elif pd.isna(x) == True:
    return ''
  elif ('МТ ' in x or 'МТ ' in x) and x[-2] != ' ':
    return x[3:].replace('"','')
  elif re.search(r'^\bМТ',x) and x[-2] == ' ':
    return x[3:-2].replace('"','')
  elif not re.search(r'^\bМТ',x) and x[-2] == ' ':
    return x[:-2].replace('"','')
  else:
    return x

df['Механизм трансформации'] = df['Механизм трансформации'].apply(mechanism)

In [87]:
df['Механизм трансформации'].unique()

array(['', 'Аккордеон', 'Пантограф', 'Еврокнижка', 'Кресла без механизма',
       'Седафлекс', 'Дельфин'], dtype=object)

Убираем из примечаний и особых отметок "отказы" и "ошибочные"

In [89]:
df['Примечание'] = df['Примечание'].fillna('-')
df['Особые отметки'] = df['Особые отметки'].fillna('-')
df = df[(df['Примечание'].str.lower().str.contains('отказ', regex=True) == False)&(df['Примечание'].str.lower().str.contains('ошибочн', regex=True) == False)\
        &(df['Особые отметки'].str.lower().str.contains('отказ', regex=True) == False)|(df['Особые отметки'].str.lower().str.contains('ошибочн', regex=True) == False)]

Меняем бренды

In [90]:
#Меняю бренды на MOON или MOONFAMILY
def brand(x):
  if x == '' or pd.isna(x) == True:
    return ''
  elif  'marketpl' in x.lower() and 'moonfamily' not in x.lower():
    return 'MOON'
  elif 'moonfamily' in x.lower() and 'marketpl' in x.lower():
    return 'MOONFAMILY'
  elif  x.lower() == 'marketplace':
    return 'MOONFAMILY'
  else:
    return x

df['Бренд'] = df['Бренд'].apply(brand)
df = df[df['Бренд'].str.lower() != 'hoff'] #Удаляю строки с брендом ХОФФ

In [91]:
df['Бренд'].unique()

array(['', 'MOON', 'MOONFAMILY'], dtype=object)

Удаляем пустую или нулевую сумму заказа

In [92]:
df = df[(df['Сумма'] != 0) & (pd.isna(df['Сумма']) == False)]

In [93]:
df['Канал продаж'].unique()

array(['Франчайзи М', 'Выставочный зал фабрики MF', 'Розница MF',
       'Розница M', 'Выставочный зал фабрики MOON', 'КиМ', 'Оптовый',
       'Франчайзи MF', 'Выставочный зал Ногинск(интернет)', 'ИМ Партнеры',
       'ИМ Регионы', 'ИМ Фабрики (Москва)'], dtype=object)

1. Комплектация "41 " - Механизм трансформации "Бельевой короб"
2. Комплектация '41.','42.',"Т3","Т10" - Пантограф
3. Комплектация 50. и 60. - Форвард
4. Комплектация "Д2" - Дельфин
5. Комплектация Н10, Н20, Н30, Н12 - Аккордеон
6. Комплектация А10, А2, А3 - Еврокнижка
7. Комплектация содержит 12 - Кресла без механизма

In [22]:
#Задаю списки названий которые будут использоваться для замены
pant = ['41.','42.','Т3','Т10']
forw = ['50.','60. ']
accord = ['Н10', 'Н20', 'Н30', 'Н12']
eurobook = ['А10', 'А2', 'А3']


df['Комплектация'] = df['Комплектация'].str.replace('H10','Н10').str.replace('H20','Н20').str.replace('H30','Н30')\
  .str.replace('H12','Н12')
df['Комплектация'] = df['Комплектация'].str.replace('A10','А10').str.replace('A2','А2').str.replace('A3','А3')
df['Комплектация'] = df['Комплектация'].str.replace('T3','Т3').str.replace('T10','Т10')

#Проводим замену, в случае если элемент из списка встречается в комплектации - присваиваем мех. трансформации
def comp41(x):
  if x['Комплектация'] == '' or pd.isna(x['Комплектация']) == True:
    return x['Механизм трансформации']
  elif '41 ' in x['Комплектация'] and x['Механизм трансформации'] == '':
    return 'Бельевой короб'
  elif any([i in x['Комплектация'] for i in pant]) and x['Механизм трансформации'] == '':
    return 'Пантограф'
  elif any([i in x['Комплектация'] for i in forw]) and x['Механизм трансформации'] == '':
    return 'Форвард'
  elif 'Д2' in x['Комплектация'] and x['Механизм трансформации'] == '':
    return 'Дельфин'
  elif any([i in x['Комплектация'] for i in accord]) and x['Механизм трансформации'] == '' \
                 and '41Н10' not in x['Комплектация'] and '41Н20' not in x['Комплектация'] and '41Н30' not in x['Комплектация']:
    return 'Аккордеон'
  elif any([i in x['Комплектация'] for i in eurobook]) and x['Механизм трансформации'] == '':
    return 'Еврокнижка'
  elif '12' in x['Комплектация'] and x['Механизм трансформации'] == '':
    return 'Кресла без механизма'
  else:
    return x['Механизм трансформации']

df['Механизм трансформации'] = df.apply(comp41, axis = 1)

Добавляем новые столбцы (Диз+мех, Спальное место, Тип дивана, Настил)(ДОДЕЛАТЬ)

In [23]:
df['Диз+мех'] = df['Дизайн'] + ' ' + df['Механизм трансформации']

In [33]:
df[df['Диз+мех'] == '043 Еврокнижка']

,Источник_строк,Акции,Банковские продукты,Бренд,Дата,Дизайн,Договор,Заказчик,Канал продаж,Кат.,...,Ткань,Цвет ткани,Цена,Шт.,Юр. лицо,Шт. для MOON,Диз+мех,Спальное место,Тип дивана,Настил
7629,Розничный отчет,NaN,NaN,MOONFAMILY,2023-07-26,043,ДЗФ-42,Багинский Сергей Валерьевич,Франчайзи М,,...,74-131,Желтый,19990.0,1,ИП Перцева С.Б.,,043 Еврокнижка,160x200,прямой,ППУ
7630,Розничный отчет,NaN,NaN,MOONFAMILY,2023-07-03,043,ДЗФ-47,Одинцова Татьяна Александровна,Франчайзи MF,,...,74-130,Бежевый,21980.0,1,ИП Черемискина Е.А.,,043 Еврокнижка,160x200,прямой,ППУ
7631,Розничный отчет,NaN,NaN,MOONFAMILY,2023-07-03,043,ДЗФ-71,Денищук Диана Владимировна,Франчайзи MF,,...,74-4,Серый,34078.0,1,"ООО ""МебельХоум""",,043 Еврокнижка,160x200,прямой,ППУ
7632,Розничный отчет,NaN,NaN,MOONFAMILY,2023-07-05,043,ДЗФ-110,Потапов Максим,Оптовый,,...,74-135,Серо-коричневый,21990.0,1,ИП Соколов А.В.,,043 Еврокнижка,160x200,прямой,ППУ
7633,Розничный отчет,NaN,NaN,MOONFAMILY,2023-07-08,043,ДЗФ-142,OOO Сенах цйцва фафА,Франчайзи М,,...,74-132,Зеленый,19990.0,1,"ООО ""Сенах""",,043 Еврокнижка,160x200,прямой,ППУ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23747,Фабричный отчет,NaN,,MOONFAMILY,2023-07-02,043,2156454,,ИМ Партнеры,2.0,...,74-129,Синий,19490.0,1,,,043 Еврокнижка,160x200,прямой,ППУ
23748,Фабричный отчет,NaN,,MOONFAMILY,2023-07-03,043,2156791,,ИМ Регионы,2.0,...,74-129,Синий,19990.0,1,,,043 Еврокнижка,160x200,прямой,ППУ
23749,Фабричный отчет,NaN,,MOONFAMILY,2023-07-06,043,2160225,,ИМ Регионы,2.0,...,74-132,Зеленый,19990.0,1,,,043 Еврокнижка,160x200,прямой,ППУ
23750,Фабричный отчет,NaN,,MOONFAMILY,2023-07-10,043,2163850,,ИМ Регионы,2.0,...,29-829,Бежевый,22990.0,1,,,043 Еврокнижка,160x200,прямой,ППУ


#Спальное место

Если Комплектация А10 х2 то Спальное место "160x200"

1. А21,A22,H10,H12 -  80x200
2. А27,A30,A32,H32,H32C -  160x200
3. Н20,H22 -  120x200
4. Н30 -  140x200
5. H32XL - 180x200




In [25]:
#Задаю списки названий которые будут использоваться для замены
eight = ['А21','А22','Н10','Н12']
six = ['А27','А30','А32','Н32C','Н32']
twelve = ['Н20','Н22']

#Задаем размеры спального места в зависимости от комплектации
def place(x):
  if x['Комплектация'] == '':
    return x['Спальное место']
  elif x['Комплектация'].count('А10') > 1 and x['Спальное место'] == '':
    return '160x200'
  elif  any([i in x['Комплектация'] for i in eight]) and x['Спальное место'] == '':
    return '80x200'
  elif  any([i in x['Комплектация'] for i in twelve]) and x['Спальное место'] == '':
    return '120x200'
  elif  'Н30' in x['Комплектация'] and x['Спальное место'] == '':
    return '140x200'
  elif  any([i in x['Комплектация'] for i in six]) and x['Спальное место'] == '':
    return '160x200'
  elif  'Н32XL' in x['Комплектация'] and x['Спальное место'] == '':
    return '180x200'
  else:
    return x['Спальное место']


df['Спальное место'] = df.apply(place, axis = 1)

Модель
1. 044, 123, 134 – 140x200
2. 110, 018, 109, 125, 127, 015 – 150x208
3. 107 – 150x210
4. 117 – 148x208

In [26]:
four = ['044','123','134'] #размер 140х200
five = ['110','018','109','125','127','015'] #размер 150х208

def place(x):
  if x['Дизайн'] == '':
    return x['Спальное место']
  elif  any([i in x['Дизайн'] for i in four]) and x['Спальное место'] == '':
    return '140x200'
  elif  any([i in x['Дизайн'] for i in five]) and x['Спальное место'] == '':
    return '150x208'
  elif  '107' in x['Дизайн'] and x['Спальное место'] == '':
    return '150x210'
  elif  '117' in x['Дизайн'] and x['Спальное место'] == '':
    return '148x208'
  else:
    return x['Спальное место']


df['Спальное место'] = df.apply(place, axis = 1)

Пантографы

1.   135,139,140,141,142,143,144,145,146,173,174,183,184,185,180 – 140x200
2.   165, 171, 172, 175, 179 - 150x200
3.   007, 008,808, 015, 107, 111, 112, 160, 018, 109, 124, 125, 161, 162, 166 – 160x200

Пантограф

1.  Модель 160 и Т30 - 160x205
2.  Модель 176,178 если Т30 и 140 - 140x200
3.  если 150 - 150x200
4.  если 005, если А10 x2 - 160x200
5. если Т30 и 150 - 150x200 (перекрывается 3 пунктом)

Модель
1. Модель 032 - 80x190

In [27]:
four = ['135','139','140','141','142','143','144','145','146','173','174','183','184','185','180'] #размер 140х200
five = ['165','171','172','175','179'] #размер 150х200
six = ['007','008','808','015','107','111','112','160','018','109','124','125','161','162','166'] #размер 160х200

def place(x):
  if x['Дизайн'] == '':
    return x['Спальное место']
  elif  any([i in x['Дизайн'] for i in four]) and x['Спальное место'] == '' and x['Механизм трансформации'] == 'Пантограф':
    return '140x200'
  elif  any([i in x['Дизайн'] for i in five]) and x['Спальное место'] == '' and x['Механизм трансформации'] == 'Пантограф':
    return '150x200'
  elif  any([i in x['Дизайн'] for i in six]) and x['Спальное место'] == '' and x['Механизм трансформации'] == 'Пантограф':
    return '160x200'

  elif  '160' in x['Дизайн'] and 'Т30' in x['Комплектация'] and x['Спальное место'] == '' and x['Механизм трансформации'] == 'Пантограф':
    return '160x205'
  elif  ('176' in x['Дизайн'] or '178' in x['Дизайн'] or '140' in x['Дизайн']) and 'Т30' in x['Комплектация']\
   and x['Спальное место'] == '' and x['Механизм трансформации'] == 'Пантограф':
    return '140x200'
  elif  '150' in x['Дизайн'] and x['Спальное место'] == '' and x['Механизм трансформации'] == 'Пантограф':
    return '150x200'
  elif  ('005' in x['Дизайн'] or x['Комплектация'].count('А10') > 1 or 'Т10' in x['Комплектация'])  and  x['Спальное место'] == '' \
   and x['Механизм трансформации'] == 'Пантограф':
    return '160x200'
  elif  '032' in x['Дизайн'] and x['Спальное место'] == '':
   return '80x190'
  else:
    return x['Спальное место']


df['Спальное место'] = df.apply(place, axis = 1)

Аккордеоны
Все комплектации, где есть модули 7Я или 91,90,17 – угловой
Остальные – прямой


In [28]:
def corner(x):
  if x['Механизм трансформации'] != 'Аккордеон':
    return x['Тип дивана']
  elif x['Механизм трансформации'] == 'Аккордеон' and \
   (bool(re.search(r'(?<!\d)90(?!\d)',x['Комплектация'])) or bool(re.search(r'(?<!\d)91(?!\d)',x['Комплектация'])) \
    or bool(re.search(r'(?<!\d)17(?!\d)',x['Комплектация'])) or bool(re.search(r'(?<!\d)7я(?!\d)',x['Комплектация'].lower()))) and x['Тип дивана'] == '':
    return 'угловой'
  elif x['Механизм трансформации'] == 'Аккордеон' and x['Тип дивана'] == '':
    return 'прямой'

df['Тип дивана'] = df.apply(corner, axis=1)

Дельфины
Комплектации с Д - угловой
Кроме Д20 и 117 комплектаций


In [29]:
def dolphin(x):
  if x['Механизм трансформации'] != 'Дельфин':
    return x['Тип дивана']
  elif x['Механизм трансформации'] == 'Дельфин' and not (bool(re.search(r'(?<!\d)д20(?!\d)',x['Комплектация'].lower())) or \
                                                        bool(re.search(r'(?<!\d)117(?!\d)',x['Комплектация']))) and x['Тип дивана'] == '':
    return 'угловой'
  elif x['Механизм трансформации'] == 'Дельфин' and x['Тип дивана'] == '':
    return 'прямой'

df['Тип дивана'] = df.apply(dolphin, axis=1) #что значит комплектация с Д

Еврокнижка
Если дизайн 118 и 7Я, 7Я - угловой,
А27 - угловой
Модели 039, 074 – прямые

005,007, 008, 160: комплектации с модулями Т7 7Я,7ПЯ,7ЛЯ, 76П(Л), 75П(Л) (не путать с 17Я) и 90 – угловые,
остальное – прямые

In [30]:
lst = ['005','007','008','160']


def eurobook(x):
  if x['Механизм трансформации'] != 'Еврокнижка':
    return x['Тип дивана']
  elif x['Механизм трансформации'] == 'Еврокнижка' and (bool(re.search(r'(?<!\d)7я(?!\d)',x['Комплектация'].lower())) or \
                                                        bool(re.search(r'(?<!\d)а27(?!\d)',x['Комплектация'].lower()))) and x['Тип дивана'] == '':
    return 'угловой'
  elif x['Механизм трансформации'] == 'Еврокнижка' and x['Дизайн'] == '118' and bool(re.search(r'(?<!\d)7я(?!\d)',x['Комплектация'].lower())) and x['Тип дивана'] == '':
    return 'угловой'
  elif x['Механизм трансформации'] == 'Еврокнижка' and x['Дизайн'] == '039' and x['Тип дивана'] == '':
    return 'прямой'
  elif x['Механизм трансформации'] == 'Еврокнижка' and x['Дизайн'] == '074' and x['Тип дивана'] == '':
    return 'прямой'
  elif x['Механизм трансформации'] == 'Еврокнижка' and any([i in x['Дизайн'] for i in lst]) and \
      (bool(re.search(r'(?<!\d)т7(?!\d)',x['Комплектация'].lower()))\
       or bool(re.search(r'(?<!\d)90(?!\d)',x['Комплектация'])) \
       or bool(re.search(r'(?<!\d)7я(?!\d)',x['Комплектация'].lower()))\
       or bool(re.search(r'(?<!\d)7пя(?!\d)',x['Комплектация'].lower()))\
       or bool(re.search(r'(?<!\d)7ля(?!\d)',x['Комплектация'].lower()))\
       or bool(re.search(r'(?<!\d)76п(?!\d)',x['Комплектация'].lower()))\
       or bool(re.search(r'(?<!\d)76л(?!\d)',x['Комплектация'].lower()))\
       or bool(re.search(r'(?<!\d)75п(?!\d)',x['Комплектация'].lower()))\
       or bool(re.search(r'(?<!\d)75л(?!\d)',x['Комплектация'].lower())))\
       and x['Тип дивана'] == '':
    return 'угловой'
  elif x['Механизм трансформации'] == 'Еврокнижка' and x['Тип дивана'] == '':
    return 'прямой'

df['Тип дивана'] = df.apply(eurobook, axis=1)

Пантографы
007, 008, 160, 161, 162, 166 с модулями Т7 и 90,91 – угловые, остальное – прямые

In [31]:
lst = ['007','008','160','161','162','166']

def func3(x):
  if x['Механизм трансформации'] != 'Пантограф':
    return x['Тип дивана']
  elif x['Механизм трансформации'] == 'Пантограф' and any([i in x['Дизайн'] for i in lst]) and \
        (bool(re.search(r'(?<!\d)Т7(?!\d)',x['Комплектация']))\
        or bool(re.search(r'(?<!\d)90(?!\d)',x['Комплектация'])) \
        or bool(re.search(r'(?<!\d)91(?!\d)',x['Комплектация']))) and x['Тип дивана'] == '':
    return 'угловой'
  elif x['Механизм трансформации'] == 'Пантограф' and x['Тип дивана'] == '':
    return 'прямой'

df['Тип дивана'] = df.apply(func3, axis=1)

Проставить настил.
Все комплектации с Lux – на НПБ, остальное – ППУ.


In [32]:
df['Настил'] = df['Комплектация'].apply(lambda x: 'НПБ' if 'lux' in x.lower() else 'ППУ')

In [ ]:
#должны остаться только эти столбцы
df = df[['Источник_строк','Договор','Дата','Заказчик','Категория заказчика','Канал продаж','Розничная точка','Дизайн','Диз+мех','Механизм трансформации',
'Комплектация','Спальное место','Тип дивана','Настил','Ткань','Цвет ткани','Кат.','Бренд','Сумма','Цена','Шт.','Шт. для MOON',
'Руководитель группы','Регион','Юр. лицо','Название салона','Скид. прод.','Акции','Продавец','Пл.отгруз','Примечание','Особые отметки',
'Банковские продукты','Коллекция','Скид. пост.','Скид. фабр.',]]

In [ ]:
#Сцепить Канал продаж, номер договора, розничная точка - через пробел, название столбца "Шт. для MOON"
df['Шт. для MOON'] = df['Канал продаж'] + ' ' + df['Договор'] + ' ' + df['Розничная точка']

<ipython-input-454-996fa1c3563e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Шт. для MOON'] = df['Канал продаж'] + ' ' + df['Договор'] + ' ' + df['Розничная точка']


In [ ]:
%%time
df.to_excel('res.xlsx', index=False)

CPU times: user 27.5 s, sys: 364 ms, total: 27.9 s
Wall time: 28.1 s
